In [ ]:
from datetime import datetime

# Базовый класс банковского счёта
class Account:
    def __init__(self, account_holder: str, balance: float = 0):
        # Проверка: начальный баланс не может быть отрицательным
        if balance < 0:
            raise ValueError("Начальный баланс не может быть отрицательным")

        # Имя владельца счёта
        self.holder = account_holder
        # Приватный атрибут для хранения баланса
        self._balance = balance
        # История операций
        self.operations_history = []

    def _record_operation(self, operation_type: str, amount: float, status: str):
        # Фиксирует операцию в истории
        operation = {
            "type": operation_type,
            "amount": amount,
            "datetime": datetime.now(),
            "balance_after": self._balance,
            "status": status
        }
        self.operations_history.append(operation)

    def deposit(self, amount: float):
        # Пополнение счёта
        if amount <= 0:
            self._record_operation("deposit", amount, "fail")
            return False

        self._balance += amount
        self._record_operation("deposit", amount, "success")
        return True

    def withdraw(self, amount: float):
        # Снятие средств
        if amount <= 0:
            self._record_operation("withdraw", amount, "fail")
            return False

        if amount > self._balance:
            self._record_operation("withdraw", amount, "fail")
            return False

        self._balance -= amount
        self._record_operation("withdraw", amount, "success")
        return True

    def get_balance(self) -> float:
        # Возвращает текущий баланс
        return self._balance

    def get_history(self):
        # Возвращает историю операций в удобном формате
        return [
            f"{op['datetime']} | {op['type']} {op['amount']} | "
            f"Баланс: {op['balance_after']} | Статус: {op['status']}"
            for op in self.operations_history
        ]


# Кредитный счёт с возможностью ухода в минус
class CreditAccount(Account):
    def __init__(self, account_holder: str, balance: float = 0, credit_limit: float = 0):
        # Инициализация базового класса
        super().__init__(account_holder, balance)
        # Кредитный лимит
        self.credit_limit = credit_limit

    def withdraw(self, amount: float):
        # Снятие средств с учётом кредитного лимита
        if amount <= 0:
            self._record_operation("withdraw", amount, "fail")
            return False

        if self._balance - amount < -self.credit_limit:
            self._record_operation("withdraw", amount, "fail")
            return False

        self._balance -= amount
        used_credit = self._balance < 0
        operation = {
            "type": "withdraw",
            "amount": amount,
            "datetime": datetime.now(),
            "balance_after": self._balance,
            "status": "success",
            "used_credit": used_credit
        }
        self.operations_history.append(operation)
        return True

    def get_available_credit(self) -> float:
        # Возвращает доступные кредитные средства
        return self._balance + self.credit_limit
    

# Пример использования классов
acc = Account("Иван", 1000)
acc.deposit(500)          # Пополнение на 500
acc.withdraw(200)         # Снятие 200
print(acc.get_balance())  # Вывод текущего баланса
print(acc.get_history())  # Вывод истории операций

credit_acc = CreditAccount("Мария", 0, 1000)
credit_acc.withdraw(500)                  # Снятие 500 с использованием кредита
print(credit_acc.get_balance())           # Баланс кредитного счёта
print(credit_acc.get_available_credit())  # Доступный кредит
print(credit_acc.get_history())           # История операций кредитного счёта



1300
['2025-12-15 21:55:54.697539 | deposit 500 | Баланс: 1500 | Статус: success', '2025-12-15 21:55:54.697539 | withdraw 200 | Баланс: 1300 | Статус: success']
-500
500
['2025-12-15 21:55:54.697539 | withdraw 500 | Баланс: -500 | Статус: success']
